In [ ]:
import os
import json
import geojson 
from postgres import Postgres  

def create_geo_boundaries_table(): 
    pg = Postgres()  # Create a new database connection
    create_table_query = """
        CREATE EXTENSION IF NOT EXISTS postgis;
        CREATE TABLE IF NOT EXISTS geo_boundaries (
            id SERIAL PRIMARY KEY,
            name TEXT,
            properties JSONB,
            geom GEOMETRY(Geometry, 4326)
        );
    """
    pg.insert(create_table_query)
   
# Step 2: Load a single GeoJSON file into the database
def load_geojson_file(file_path):
    pg = Postgres()  # Create a new database connection
    # Read the GeoJSON file
    with open(file_path) as f:
        geojson_data = open(file_path).read()
    
    # Prepare the insert statement
    insert_query = """
        INSERT INTO geo_boundaries (name, properties, geom)
        VALUES (%s, %s, ST_SetSRID(ST_GeomFromGeoJSON(%s), 4326));
    """

    # Load or parse your GeoJSON data
    data = geojson.loads(geojson_data)  # or geojson.load() if reading from a file

    # Check if data is a FeatureCollection, which contains 'features'
    if isinstance(data, geojson.FeatureCollection):
        for feature in data.features:
            name = feature.get('properties', {}).get('Name', 'Unnamed')
            properties = feature.get('properties', {})
            geometry = geojson.dumps(feature.geometry)  # Convert geometry to GeoJSON string

            # Execute the insert statement with parameters
            pg.insert(insert_query, (name, json.dumps(properties), geometry))

    # Check if data is a single Feature
    elif isinstance(data, geojson.Feature):
        name = data.get('properties', {}).get('Name', 'Unnamed')
        properties = data.get('properties', {})
        geometry = geojson.dumps(data.geometry)  # Convert geometry to GeoJSON string

        # Execute the insert statement with parameters
        pg.insert(insert_query, (name, json.dumps(properties), geometry))

    # Check if data is a bare Geometry (Point, LineString, etc.)
    elif isinstance(data, (geojson.Point, geojson.LineString, geojson.Polygon,
                        geojson.MultiPoint, geojson.MultiLineString, geojson.MultiPolygon)):
        name = 'Unnamed'
        properties = {}  # No properties available for bare geometries
        geometry = geojson.dumps(data)  # Convert the geometry to a GeoJSON string

        # Execute the insert statement with parameters
        pg.insert(insert_query, (name, json.dumps(properties), geometry))

# Step 3: Loop through all GeoJSON files in the folder
def load_all_geojson_files(geo_boundaries_folder):
    # Iterate over each file in the geojson_folder
    for filename in os.listdir(geo_boundaries_folder):
        if filename.endswith(".geojson"):
            file_path = os.path.join(geo_boundaries_folder, filename)
            print(f"Loading file: {filename}")
            load_geojson_file(file_path)
    print("All GeoJSON files have been loaded into geo_boundaries.")

# Run the functions
def import_geo_boundaries(geo_boundaries_folder):
    load_all_geojson_files(geo_boundaries_folder)
    print("GeoJSON data loaded successfully into geo_boundaries.")

import_geo_boundaries('files/geo_boundaries/')  # Replace with your actual folder path


Loading file: alwoodly.geojson
All GeoJSON files have been loaded into geo_boundaries.
GeoJSON data loaded successfully into geo_boundaries.
